# Tensorflow로 GAN의 TOYMODEL 구현하기

<div style="text-align: right"><strong>YBIGTA 10기 김지중</strong></div>

지난 시간, 우리는 Generative Adversarial Model (2014, Ian Goodfellow at el.)에 대해 정리해보았습니다. 오늘은 GAN의 간단한(..?) 예제코드를 소개하려고 합니다. 아래 링크 중 "원문"에 해당하는 블로그를 참고했구요. 번역,요약 후 필요한 설명을 추가했습니다. 설명 중간중간 해당 부분에 대한 코드도 적어두었습니다. (원문 블로그에서도 그렇게 했어요.) 다만, 이 문서에 적힌 코드는 설명을 위해 간단하게 적어둔 것이므로 작동하지 않을 수도 있습니다. 정확한 코드는 아래 링크의 소스코드를 참고해주시기 바랍니다.

**Sources**

원문 블로그 - http://blog.aylien.com/introduction-generative-adversarial-networks-code-tensorflow/ <br>
원문의 참고 문헌 - http://blog.evjang.com/2016/06/generative-adversarial-nets-in.html<br>
Source code - https://github.com/AYLIEN/gan-intro/blob/master/gan.py

---

# 복습

우선 GAN에 대해 복습을 해보겠습니다. GAN이란 기본적으로 생성 모델이구요, 나름대로 정리를 해보자면 아래와 같습니다.


**similar_data = GAN(noise, real_data)**<br>
noise고 불리는 인풋값을 넣어주면, 실제 데이터와 유사한 분포를 따르는 새로운 데이터를 생성하는 모델인데요. 지난 시간에 noise는 특정한 분포(주로 유니폼 분포)를 따르도록 설정해준다고 했습니다. 유사한 데이터를 만들어내는 과정의 핵심은 D와 G라는 서로 다른 두 네트워크를 학습시키는 과정입니다.

**Discriminator(D)**<br>
similar_data와 real_data를 잘 구별하도록 학습시킵니다.

**Generator(G)**<br>
Discriminator가 실수할 확률이 높도록 학습시킵니다. 즉, 잘 학습된 분류기가 헷갈릴정도로 실제와 유사한 데이터를 생성시킵니다.

---

## TOY MODEL: 정규분포에 근사하는 데이터 생성 모델

본격적으로 TOY MODEL을 구현해봅시다. 우리가 오늘 구현해보려는 TOY MODEL은 아래와 같이 요약됩니다.

1) 실제 데이터가 N(4,0.5)의 정규분포를 따른다고 가정합니다<br>
2) Noise 값으로 U(-8,8)의 균일 분포에 랜덤한 교란을 적용합니다.<br>
3) 이 Noise값을 input으로 활용하여, N(4,0.5)와 유사한 분포를 갖는 새로운 데이터를 생성합니다. 

#### 1) 실제 데이터 생성 및 샘플링

먼저, "진짜" 데이터 분포를 만들어봅시다. 평균 4, 표준편차 0.5의 정규 분포로 만들어줍니다.N개의 샘플을 뽑아내는 sample이라는 메소드도 추가했습니다.

In [2]:
class DataDistribution(object):
    def __init__(self):
        self.mu = 4
        self.sigma = 0.5
        
    def sample(self, N):
        samples = np.random.normal(self.mu, self.sigma, N)
        samples.sort()                                         ## 굳이 오름차순으로 정렬한 이유는 모르겠습니다 ㅠㅠ  
        return samples

그 결과로 나타나는 분포의 형태는 아래와 같습니다.

<a href="" rel="some text"><img src="http://blog.aylien.com/wp-content/uploads/2016/08/data.png" alt="" /></a>

#### 2) Noise 생성 및 샘플링

이제 Generator에 넣어줄 input noise distribution을 만들어봅시다. stractified sampling 기법이 이용되는데, 이는 특정한 range의 유니폼 분포에 랜덤하게 교란값을 주는 것을 의미합니다. 마찬가지로 샘플링하는 메소드도 구현이 되었습니다.

In [3]:
class GeneratorDistribution(object):
    
    def __init__(self, range):
        self.range = range
        
    def sample(self,N):
        return np.inspace(-self.range, self.range, N) + np.random.random(N) * 0.01  # 교란값을 넣어주는 장점에는 뭐가 있을까요..

#### 3) Generator, Discriminator 구현

Generator모델과 Discriminator 네트워크는 간단하게 구축했습니다. 먼저 Generator 모델은 두 개의 layer로 이루어집니다. 구성은 아래와 같습니다.

1) 선형변환 + 활성화함수(softplus)<br>
2) 선형변환<br>

참고로, softplus는 ReLU를 매끄러운 곡선 형태로 근사한 활성화함수입니다. 수식은 아래와 같으며, 미분하면 logistic function이 됩니다.<br>

\\(f(x)=\ln[1+\exp(x)]\\)<br>

아무튼 Genrator를 구현하면 아래와 같습니다.


In [9]:
def generator(input, hidden_size):
    h0 = tf.nn.softplus(linear(input,hidden_size,"g0")) # 선형변환 후 softplus 함수 적용
    h1 = linear(h, 1, "g1")                             # 단순 선형변환 (fully connected layer)
    return h1



# 아래는 선형변환 함수입니다.

def linear(input, output_dim, scope=None, stddev=1.0):   # 가중치를 불러와 input에 선형변환을 적용하는 함수.
    with tf.variable_scope(scope or 'linear'):                 
        w = tf.get_variable(                             # 가중치 매트릭스, 매트릭스 사이즈는 (인풋사이즈 * 아웃풋사이즈)
            'w',
            [input.get_shape()[1], output_dim],
            initializer=tf.random_normal_initializer(stddev=stddev) # 표준편차가 1인 랜덤값으로 초기화
        )           
        b = tf.get_variable(                             # 편향 벡터를 만듭니다. (영벡터로 초기화)
            'b',
            [output_dim],
            initializer=tf.constant_initializer(0.0)
        )
        return tf.matmul(input, w) + b

사실, GAN에서 Generator도 중요하지만, 그보다 더 중요한건 Discriminator입니다. 백날 돌려봐야 Discriminator가 학습이 안된다면, 뭔가 유사한 output을 생성해 낼 여지가 없기 때문입니다. 그래서 Discriminator는 모델을 조금 더 깊게 만들었습니다. 가장 마지막의 레이어를 제외하고는 모두 tanh를 활성화함수로 두었습니다. 마지막 레이어의 활성화함수로는 sigmoid를 두었습니다. 

결국 이 함수는 어떤 input이 들어왔을 때 이 input이 실제 데이터일 확률을 반환합니다.

In [10]:
def discriminator(input, hidden_size):
    h0 = tf.tanh(linear(input, hidden_size*2, "d0")) # 선형변환 후 tanh 적용
    h1 = tf.tanh(linear(h0, hidden_size*2, "d1"))
    h2 = tf.tanh(linear(h1, hidden_size*2, "d2"))
    h3 = tf.sigmoid(linear(h2, 1, "d3"))             # 마지막 레이어는 sigmoid 적용
    return h3

Discriminator와 Generator의 그래프를 그려보겠습니다.각각의 loss function도 구현해보구요.<br>
지난 시간, 우리가 배운 GAN의 목적함수는 아래와 같이 표현됩니다.<br>

>\\(Argmin_G Argmax_D E[log D_1(x)]+E[log (1 - D_2(G(z)))]\\)<br>

위 목적함수를 minimize하는 G를 찾고, maximize하는 D를 찾는 것입니다. 그래서 minimax 게임이라는 개념도 다뤘었는데요.<br>
아무튼간에, 우리는 위 함수를 조금 변형해서 loss function을 찾을겁니다.<br>


D1은 Discriminator가 실제 데이터를 실제 데이터라고 판별할 확률이고, D2는 생성된 유사 데이터를 실제 데이터라고 판별할 확률입니다.따라서 Discriminator는 D1과 (1-D2)를 최대화하는 방향으로 학습되어야 합니다. log는 증가함수이므로, Discriminator는 위 목적함수를 최대화하는 방향으로 학습이 됩니다.<br>

근데 loss function이란건 loss를 계산하는 function이고, loss는 모델이 학습을 통해 줄여야 하는 값입니다. 그래서 Discriminator의 loss function은 위 식에 마이너스를 적용한 값이나 다름이 없습니다.

**Discriminator의 loss**<br>
>\\(-E[log D_1(x)]-E[log (1 - D_2(G(z)))]\\)

>= \\(E[-log D_1(x)-log (1 - D_2(G(z)))]\\)

G에 대해서 위 목적함수를 살펴봅시다. G는 위 식의 2번째 항을 minimizing하는 방식으로 진행됩니다. 어차피 minimizing할 거라면 아래와 같이 조금 더 단순한 형태로 만들 수 있겠네요.

**Generator의 loss**<br>
>= \\(E[-log D_2(G(z)]\\)

이를 코드로 구현하면 아래와 같습니다.

In [5]:
with tf.variable_scope("G"):                            # Generator를 "G"라는 이름의 변수로 생성
    z = tf.placeholder(tf.float32, shape=(None, 1))     
    G = generator(z, hidden_size)

with tf.variable_scope("D") as scope:                   # Discriminator를 "D"라는 이름의 변수로 생성 
    x = tf.placeholder(tf.float32, shape = (None, 1))
    D1 = discriminator(x, hidden_size)                  # 실제 데이터를 집어넣었을때 분류기가 실제 데이터라고 판별할 확률
    scope.reuse_variables()
    D2 = discriminator(G, hidden_size)                  # 유사 데이터를 집어넣었을때 분류기가 실제 데이터라고 판별할 확률

loss_d = tf.reduce_mean(-tf.log(D1) - tf.log(1-D2))     # Discriminator의 loss function
loss_g = tf.reduce_mean(-tf.log(D2))                    # Generator의 loss function

NameError: name 'generator' is not defined

자, 이제 네트워크의 그래프까지 그려봤구요, 학습을 시키기에 적합한 optimizer를 구현해봅시다. 간단하게 텐서플로에서 제공되는 GradientDescentOptimizer를 적용해보죠. 

learning rate는 exponential하게 줄여봅시다. 즉, 초기에는 높은 learning rate값을 통해 빠르게 학습하고, 지속적으로 learning rate를 줄여가며 파라미터가 보다 미세하게 조정이 되도록 유도해봅시다. 이러한 과정은 아래의 코드를 통해 구현이 가능합니다.

In [6]:
def optimizer(loss, var_list):
    initial_learning_rate = 0.005                       # 초기 lr = 0.005
    decay = 0.95                                        
    num_decay_steps = 150                               # 150 frame이 지나면 learning rate를 이전 단계의 95% 수준으로 조정합니다.
    batch = tf.Variable(0)
    learning_rate = tf.train.exponential_decay(
        initial_learning_rate,
        batch,
        num_decay_steps,
        decay,
        staircase = True
    )
    optimizer = GradientDecentOptimizer(learning_rate).minimize( # loss를 최소화합니다.
        loss,
        global_step = batch,
        var_list = var_list
    )
    return optimizer

    vars = tf.trainable_variables()
    d_params = [v for v in vars if v.name.startswith("D/")]   # 아까 Discriminator 변수를 D라고 했으므로, D로 시작하는 가중치들을 불러옵니다..
    g_params = [v for v in vars if v.name.startswith("G/")]   # Generator 변수들은 아까 G라고 설정했습니다.
    
    opt_d = optimizer(loss_d, d_params)
    opt_g = optimizer(loss_g, g_params)
    

이제 텐서플로의 세션을 열고 이 모델을 학습시켜봅시다. 모델링에 관해서는 지난 시간 논문 스터디 자료의 Algorithm 1을 참고해주세요

In [46]:
with tf.Session() as session:                                # tensorflow 세션을 엽니다
    tf.initialize_all_variables().run()                      # variable들을 init하구요
    for step in xrange(num_steps):                           # epoch을 돌립니다.
        x = data.sample(batch_size)                          # 원 데이터에서 batch_size만큼 샘플링하구요
        z = gen.sample(batch_size)                           # Generator에 넣어줄 노이즈를 샘플링합니다.
        
        session.run([loss_d, opt_d], {
            x: np.reshape(x, (batch_size, 1))                # batch_size * 1 형태의 매트릭스로 셰이프를 변환하고
            z: np.reshape(z, (batch_size, 1))                # D를 학습시킵니다.
        })
        
        z = gen.sample(batch_size)                           
        session.run([loss_g, opt_g], {                       # 마찬가지로 bathc_size * 1 형태의 매트릭스로 reshape 후
            z: np.reshape(z, (batch_size, 1))                # G를 학습시킵니다.
        })

SyntaxError: invalid syntax (<ipython-input-46-c2c1997c6b43>, line 9)

아래 링크는 generator가 데이터 분포에 근사하기 위해 학습하는 과정을 보여줍니다.<br>
https://www.youtube.com/watch?v=mObnwR-u8pc&feature=youtu.be

초반에는 Generator가 실제 데이터와 매우 다른 분포를 보임을 확인할 수 있습니다. 하지만 조금만 지나도 실제 데이터 분포에 가까워지는 모습을 확인할 수 있습니다. 그런데 시간이 더 지나면 실제 데이터의 평균값(mean=4) 근처에만 데이터가 생성됩니다. 모든 학습 과정을 마친 뒤 결과는 아래와 같이 나옵니다.

<a href="" rel="some text"><img src="http://blog.aylien.com/wp-content/uploads/2016/08/gan-trained-1-768x528.png" alt="" /></a>

그래프를 보면 만들어진 데이터가 원래 데이터의 평균 근처에 엄청나게 몰려있다는 걸 확인할 수 있습니다. Generator가 실제 데이터의 평균값만을 생성해도 Discriminator를 속일 확률이 매우 높기 때문입니다. 특히나 이 케이스의 경우에는 원래 데이터가 정규분포를 따르죠. 정규분포에서 평균값을 기준으로 그거보다 클 확률, 그리고 작을 확률은 각각 0.5입니다. 그러니깐 Generator가 원 데이터의 평균치만 생성해도 Discriminator가 데이터를 제대로 판별할 확률이 반반이 됩니다.

---

## Improving sample diversity

Tim Sallmans는 이러한 현상을 대표적인 실패한 GAN 모델링이라고 이야기합니다. 해결할 수 있는 방법은 없을까요? 그가 내놓은 해결책은 다음과 같습니다. Discriminator가 *minibatch discrimination* 이라는 기법을 통해 한 번에 여러 가지 샘플들을 살펴보는 능력을 부여하는 것입니다. 무슨 말인지 보다 자세히 설명해보겠습니다. 우리가 위에 만든 토이 모델의 경우 Discriminator가 하나의 input값을 받고 이 input이 실제 데이터일 확률을 반환했죠. 

이 대신 Discriminator가 여러 개의 input값을 받아서 서로 비교할 여지를 주는 것입니다. Discriminator의 input값을 여러개로 바꾸기만 해도 성능이 어느정도 향상되긴 합니다. 근데 잘 생각해봅시다. 만약 Discriminator가 minibatch 안에 유독 비슷한 샘플들이 많을 경우 이 샘플들을 "Generator가 만든 가짜 이미지"라고 판별한다면, Generator는 "너무 비슷한 데이터들을 만들어내면 안되겠구나"라고 판단하지 않을까요? Sallmans는 그의 논문 *Improved Techniques for Training GANs* 에서는 이러한 방법을 제시합니다. 미니배치 안에서 서로 다른 샘플들의 유사성을 거리로 나타내어 이 거리 정보를 Discriminator가 학습하도록 만드는 겁니다. 

<a href="" rel="some text"><img src="https://jaydottechdotblog.files.wordpress.com/2016/12/gan-minibatch-discrimination-figure1.png?w=474&h=310" alt="" /></a>

출처: Improved Techniques for Training GANs(2016), p.3 (https://arxiv.org/pdf/1606.03498.pdf)

구체적인 방법론은 아래와 같습니다.

* Discriminator에서 학습 과정 중, 중간에 있는 레이어 하나를 고르고 결과값을 가져온다.(f1, f2, ... fn)
* 이 값을 3차원의 텐서에 곱해서 행렬을 만든다.
* 이 행렬들끼리 L1-distance를 계산한다. 그리고 자연상수에 음의 지수값으로 대입한다. 
    * exp(-x)
* 이 지수들의 총합계가 이 미니배치의 feature가 된다.
* 원래 인풋 뒤에 새로 만들어진 minibatch feature를 concat한다. 그리고 Discriminator의 다음 레이어에 전달한다.

이를 텐서플로로 나타내면 아래와 같습니다.

## SOMEBODY HELP ME
* 여기 너무 어려워서 이해를 못했습니다 ㅠ

In [49]:
def minibatch(input, num_kernels = 5, kernel_dim = 3): # 거리 정보를 concat한 minibatch를 반환하는 함수입니다.
    x = linear(input, num_kernels * kernel_dim)     # input값을 사이즈가 15인 벡터로 선형변환합니다.
    activation = tf.reshape(x, (-1, num_kernels, kernel_dim)) # 3차원의 텐서로 리셰이핑합니다(배치 사이즈 * 커널 개수 * 커널 차원)
    diffs = tf.expand_dims(activation, 3)\      # element간 거리 텐서를 구하기 위해 batch_size *num_ker * ker_dim의 차원을
            - tf.expand_dims(tf.transpose(activation, [1,2,0],0))  # num_ker * ker_dim * batch_size로 transpose하고 뻅니다
    abs_diffs = tf.reduce_sum(tf.abs(diffs),2)              # "거리"니깐 위에서 구한 차에 절대값을 적용합니다.
    minibatch_features = tf.reduce_sum(tf.exp(-abs_diffs), 2)  # 거리를 자연상수에 음의 지수로 집어넣습니다. (이게 feature)
    return tf.concat(1, [input, minibatch_features],1)      # feature를 추가한 minibatch를 리턴합니다.

토이모델에 이러한 minibatch discrimination 방법론을 적용해보았습니다. 이 모델에서의 학습 과정은 아래의 영상과 같이 나타났습니다.

https://youtu.be/0r3g7-4bMYU

이제 만들어진 데이터 분포의 폭이 원래 분포와 유사하게 나옴을 확인할 수가 있는데요, 분포가 수렴한 뒤의 형태는 아래의 그래프와 같습니다.

<a href="" rel="some text"><img src="http://blog.aylien.com/wp-content/uploads/2016/08/gan-trained-2-1-768x528.png" alt="" /></a>

하나 짚고 넘어야 하는 점은 이 과정에서 "배치 사이즈" 역시 중요한 하이퍼파라미터가 된다는 점입니다. 이 문서에서 사용된 토이모델에서는 배치 사이즈를 16 이하의 작은 숫자를 적용하였습니다.